In [ ]:
!pip install -U fashion-clip

In [26]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

In [27]:
fclip = FashionCLIP('fashion-clip')

In [28]:
# 한번에 32개 임베딩 
images = [] # 예제에서는 파일로 불러옴
image_embeddings = fclip.encode_images(images, batch_size=32)

IndexError: list index out of range

In [31]:
# 테스트 용 이미지 
img = Image.open('data_for_fashion_clip/108775044.jpg')
images = [img]

In [32]:
embeded_cloth = fclip.encode_images(images,batch_size=1)

100%|██████████| 1/1 [00:00<00:00,  2.25it/s]


In [58]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="all_clothes")
collection = chroma_client.get_collection(name="all_clothes")

UniqueConstraintError: Collection all_clothes already exists

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        # embed the documents somehow
        
        
        # return embeddings
        pass

In [51]:
type(embeded_cloth[0])

numpy.ndarray

In [56]:
# ids < 꼭 채워야 함
# embeded_cloth[0]
embeded_list2 = [float(x) for x in embeded_cloth[0]]

In [57]:
collection.add(
    # documents=["doc1"],
    embeddings=[embeded_list2],
    metadatas=[{"chapter": "3", "verse": "16"}],
    ids=["id1"] 
)

In [ ]:
# 코사인 유사도 대신 내적을 사용하기 위한 표준화(normalize) 
image_embeddings = image_embeddings/np.linalg.norm(image_embeddings, ord=2, axis=-1, keepdims=True)

In [ ]:
# 텍스트 임베딩 예제
text_embedding = fclip.encode_text(["a pair of pink shorts"], 32)[0]

id_of_matched_object = np.argmax(text_embedding.dot(image_embeddings.T))

found_object = subset["article_id"].iloc[id_of_matched_object].tolist()

fixed_height = 224

image = Image.open(f"data_for_fashion_clip/{found_object}.jpg")
height_percent = (fixed_height / float(image.size[1]))
width_size = int((float(image.size[0]) * float(height_percent)))
image = image.resize((width_size, fixed_height), Image.NEAREST)

image